In [1]:
!pip install pycombo
!pip install fastnode2vec

     |████████████████████████████████| 136 kB 4.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 213 kB 33.9 MB/s 
  Using cached importlib_metadata-1.7.0-py2.py3-none-any.whl (31 kB)
  Created wheel for pycombo: filename=pycombo-0.1.7-cp37-cp37m-manylinux_2_27_x86_64.whl size=98830 sha256=9e19beca46226f3c9742a00b868b377976ed462204d0dc244d4b1c47ed62f386
  Stored in directory: /root/.cache/pip/wheels/58/52/18/4c1b80cd45c091e2c1ea442729343ac984dc66b3a678e2c251
Successfully built pycombo
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.3
    Uninstalling importlib-metadata-4.11.3:
      Successfully uninstalled importlib-metadata-4.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdo

In [2]:
import networkx as nx
import numpy as np
import time
from sklearn.cluster import KMeans

import pycombo
import fastnode2vec

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)

In [3]:
nx.__version__

'2.6.3'

In [4]:
def modularity_matrix(adj):
    w_in = adj.sum(dim=0, keepdim=True)
    w_out = adj.sum(dim=1, keepdim=True)
    T = w_out.sum()
    Q = adj / T - w_out * w_in / T ** 2
    return Q

def modularity(Q, partition):
    return (Q * (partition.reshape(-1,1) == partition.reshape(1,-1))).sum()

def residential_page_rank_embedding(A, alpha=0.85):
    A = A / A.sum(axis = 1)
    n = A.shape[0]
    AI = np.linalg.inv(np.eye(n) - A * alpha)
    X = (1 - alpha) * AI
    return X.transpose()

def node2vec_embedding(G: nx.Graph, dim=10, walk_length=100, context=10, p=2.0, q=0.5, workers=2, seed=42):
    if nx.is_weighted(G):
        n2v_graph = fastnode2vec.Graph([(str(edge[0]), str(edge[1]), edge[2]['weight']) for edge in G.edges(data=True)],
                directed=False, weighted=True)
    else:
        n2v_graph = fastnode2vec.Graph([(str(edge[0]), str(edge[1])) for edge in G.edges(data=True)],
                    directed=False, weighted=False)
    n2v = fastnode2vec.Node2Vec(n2v_graph, dim=dim, walk_length=walk_length, context=context, p=p, q=q, workers=workers, seed=seed)
    n2v.train(epochs=100)
    n2v_embeddings = np.array([n2v.wv[str(node)] for node in G])
    return n2v_embeddings

def rpr_clustering(A: np.array, n_clusters=4, kmeans_runs=100, alpha=0.85, seed=42):
    rpr_embedding = residential_page_rank_embedding(A, alpha)
    rpr_cluster_labels = KMeans(n_clusters=n_clusters, n_init=kmeans_runs, random_state=seed).fit(rpr_embedding).labels_
    return rpr_cluster_labels

def n2v_clustering(G: nx.Graph, n_clusters=4, kmeans_runs=100, dim=10, walk_length=100, context=10, p=2.0, q=0.5, workers=2, seed=42):
    n2v_embeddings = node2vec_embedding(G, dim=dim, walk_length=walk_length, context=context, p=p, q=q, workers=workers, seed=seed)
    n2v_cluster_labels = KMeans(n_clusters=n_clusters, n_init=kmeans_runs, random_state=seed).fit(n2v_embeddings).labels_
    return n2v_cluster_labels

In [7]:
Gs =  [nx.from_numpy_array(nx.to_numpy_array(nx.karate_club_graph(), weight=None)), nx.les_miserables_graph()]
for SEED in range(3):
    for G in Gs:
        print("------------------------------------")
        print("SEED = ", SEED)
        print("Processing graph...", G.name)
        combo_comms, combo_mod = pycombo.execute(G)
        n_comms = np.unique(list(combo_comms.values())).size
        A = nx.to_numpy_array(G)
        adj = torch.FloatTensor(A)
        Q = modularity_matrix(adj)
        rpr_C = rpr_clustering(A, n_clusters=n_comms, seed=SEED)
        n2v_C = n2v_clustering(G, n_clusters=n_comms, seed=SEED)
        print('combo:', combo_mod, ', resid. page rank k-means:', modularity(Q, rpr_C).item(), ', node2vec kmeans:', modularity(Q, n2v_C).item())

------------------------------------
SEED =  0
Processing graph... 


Training: 100%|██████████| 3400/3400 [00:02<00:00, 1603.01it/s]


combo: 0.41978961209730403 , resid. page rank k-means: 0.4197896122932434 , node2vec kmeans: 0.39340895414352417
------------------------------------
SEED =  0
Processing graph... 


Training: 100%|██████████| 7700/7700 [00:02<00:00, 3259.50it/s]


combo: 0.566687983343249 , resid. page rank k-means: 0.5447605848312378 , node2vec kmeans: 0.5652743577957153
------------------------------------
SEED =  1
Processing graph... 


...

combo: 0.566687983343249 , resid. page rank k-means: 0.5447605848312378 , node2vec kmeans: 0.5422798991203308
------------------------------------
SEED =  2
Processing graph... 


Training: 100%|██████████| 3400/3400 [00:00<00:00, 4284.77it/s]


combo: 0.41978961209730403 , resid. page rank k-means: 0.4197896122932434 , node2vec kmeans: 0.4197896122932434
------------------------------------
SEED =  2
Processing graph... 


Training: 100%|██████████| 7700/7700 [00:01<00:00, 4644.76it/s]


combo: 0.566687983343249 , resid. page rank k-means: 0.5447605848312378 , node2vec kmeans: 0.5382205247879028


In [8]:
class GNNLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.0):
        super(GNNLayer, self).__init__()
        self.weight1 = nn.Parameter(torch.randn(in_features, out_features)) # 0.5 * torch.eye(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(in_features, out_features)) # -0.5 * torch.ones(1, out_features))
        self.dropout = dropout

    def forward(self, input):
        v1 = torch.mm(input, self.weight1)
        output = v1 + self.bias
        output = F.dropout(output, p=self.dropout, training=self.training)
        return output

class GNN_MLP(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.0):
        super(GNN_MLP, self).__init__()
        self.n_layers = 1
        self.hidden_dim = 8
        if self.n_layers > 1:
            layers = [GNNLayer(in_features, self.hidden_dim, dropout)]
        else:
            layers = [GNNLayer(in_features, out_features, dropout)]
        for _ in range(self.n_layers-2):
            layers.append(GNNLayer(self.hidden_dim, self.hidden_dim, dropout))
        if self.n_layers > 1:
            layers.append(GNNLayer(self.hidden_dim, out_features, dropout))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for i in range(self.n_layers - 1):
            x = self.layers[i](x)
            x = nn.ReLU(x)
        x = self.layers[-1](x)
        x = nn.Softmax(dim=1)(x)
        #x = 1.0 + x - x.max(dim=-1, keepdim=True).values
        #x = torch.clamp(x, 0, 1)
        #x = x / x.sum(dim=-1, keepdim=True) #normalize st sum = 1
        return x

In [11]:
for SEED in range(3):
    for G in Gs:
        print("------------------------------------")
        print("SEED = ", SEED)
        print("Processing graph...", G.name)
        combo_comms, combo_mod = pycombo.execute(G)
        n_comms = np.unique(list(combo_comms.values())).size
        A = nx.to_numpy_array(G)
        adj = torch.FloatTensor(A)
        Q = modularity_matrix(adj)
        
        rpr_embedding = residential_page_rank_embedding(A, 0.85)
        features = torch.FloatTensor(rpr_embedding)
        best_best_mod = -1
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        t_total = time.time()
        model = GNN_MLP(features.shape[1], n_comms + 2)
        lr = 0.002
        n_epochs = 6000
        optimizer = optim.Adam(model.parameters(), lr=lr)
        for epoch in range(n_epochs):
            t_1run = time.time()
            optimizer.zero_grad()
            out_embed = model(features)
            C = out_embed#[:, :n_comm]
            Q1 = torch.mm(C.T, Q)
            Q2 = torch.mm(Q1, C)
            loss = torch.trace(Q2)
            loss = -loss
            loss.backward()
            optimizer.step()
            if epoch == 0 or loss < best_loss:
                best_loss = loss #- torch.trace(Q)
                best_C = C.data
                best_embed = out_embed.data
                best_epoch = epoch
            if n_epochs <= 20 or epoch % (n_epochs//20) == 0 or epoch == n_epochs - 1:
                #optimizer = optim.Adam(model.parameters(), lr=lr)
                print('Epoch: {:04d}'.format(epoch + 1),
                        'Modularity: {:.8f}'.format(-best_loss.item()),
                        'time: {:.4f}s'.format(time.time() - t_1run))
        print("Optimization Finished!")
        print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
        print(best_loss)
        best_best_mod = max(best_best_mod, -best_loss.item())
        #print(best_embed)
    print(best_best_mod)

------------------------------------
SEED =  0
Processing graph... 
Epoch: 0001 Modularity: -0.00834351 time: 0.0015s
Epoch: 0301 Modularity: 0.11897559 time: 0.0009s
Epoch: 0601 Modularity: 0.29895350 time: 0.0010s
Epoch: 0901 Modularity: 0.36741373 time: 0.0011s
Epoch: 1201 Modularity: 0.38858986 time: 0.0009s
Epoch: 1501 Modularity: 0.39584810 time: 0.0011s
Epoch: 1801 Modularity: 0.39941776 time: 0.0006s
Epoch: 2101 Modularity: 0.40150204 time: 0.0006s
Epoch: 2401 Modularity: 0.40284574 time: 0.0006s
Epoch: 2701 Modularity: 0.40376967 time: 0.0006s
Epoch: 3001 Modularity: 0.40443480 time: 0.0006s
Epoch: 3301 Modularity: 0.40492997 time: 0.0008s
Epoch: 3601 Modularity: 0.40530834 time: 0.0015s
Epoch: 3901 Modularity: 0.40560350 time: 0.0010s
Epoch: 4201 Modularity: 0.40583745 time: 0.0006s
Epoch: 4501 Modularity: 0.40602544 time: 0.0006s
Epoch: 4801 Modularity: 0.40617812 time: 0.0006s
Epoch: 5101 Modularity: 0.40630323 time: 0.0006s
Epoch: 5401 Modularity: 0.40640661 time: 0.0006s
